### Uising Cedar API from Python
This notebook demonstrates an ability to use the Cedar API from Python.
Tools and packages used:

* python 3.5.1 (any Python 3+ should work)
* requests package (`pip install requests` should work)
* Local file `$HOME\.cedarkey`

In [1]:
import os
import pathlib
import requests

resource_endpoint = 'https://resource.metadatacenter.org'
terminology_endpoint = 'https://terminology.metadatacenter.org/bioportal'

Load apiKey from file

In [2]:
cedarkeypath = os.path.join(os.environ['HOME'], '.cedarkey')
cedarkeypath

'C:\\Users\\davisda4\\.cedarkey'

In [3]:
with open(cedarkeypath) as fp:
    apikey = fp.read().strip()

Create a session holding the authorization header and accepting JSON

In [4]:
sess = requests.Session()
sess.headers['Authorization'] = 'apiKey %s' % apikey
sess.headers['Accept'] = 'application/json'

### Resource Service
Do a search on the resource service

In [5]:
search_params = {
    'q': 'LC Movie Example',
    'resource_types': 'template',
    'sharing': 'shared-with-me',
}
response = sess.get(resource_endpoint + '/search', params=search_params)
response

<Response [200]>

In [6]:
data = response.json()
len(data['resources'])

10

__Issue 1:__ API worked, but too many results found. Paging required to find the one with title "LC Movie Example"

No more matter - can we find the one we need?

In [7]:
offset = 0
search_params = {
    'q': 'LC Movie Example',
    'resource_types': 'template',
    'sharing': 'shared-with-me',
    'limit': 10,
}
templates = []
total = 1
while offset < total:
    search_params['offset'] = offset   
    response = sess.get(resource_endpoint + '/search', params=search_params)
    assert response.ok
    data = response.json()
    total = data['totalCount']
    offset += len(data['resources'])
    templates += data['resources']
    
len(templates)

10

Oops - no more pages - 10 is all there is. So, is it there?

In [8]:
matched=[template for template in templates if template['schema:name']=='LC Movie Example']
matched


[{'@id': 'https://repo.metadatacenter.org/templates/916d64a9-e878-44e2-9d69-69eaf0bad233',
  'activeUserCanRead': True,
  'bibo:status': 'bibo:draft',
  'createdByUserName': 'Rafael Goncalves',
  'isLatestDraftVersion': None,
  'isLatestPublishedVersion': None,
  'isLatestVersion': None,
  'lastUpdatedByUserName': 'Rafael Goncalves',
  'nodeType': 'template',
  'oslc:modifiedBy': 'https://metadatacenter.org/users/2c81c03c-c905-4a46-8126-9e378f5c25bb',
  'ownedBy': 'https://metadatacenter.org/users/2c81c03c-c905-4a46-8126-9e378f5c25bb',
  'ownedByUserName': 'Rafael Goncalves',
  'pav:createdBy': 'https://metadatacenter.org/users/2c81c03c-c905-4a46-8126-9e378f5c25bb',
  'pav:createdOn': '2018-02-01T15:03:46-0800',
  'pav:lastUpdatedOn': '2018-02-01T16:42:32-0800',
  'pav:version': '0.0.1',
  'schema:description': None,
  'schema:identifier': None,
  'schema:name': 'LC Movie Example'},
 {'@id': 'https://repo.metadatacenter.org/templates/ef789e1c-324f-474e-8457-69a4215117b0',
  'activeUser

Oh, yeah, I made a copy - we can have a simpler example next time.   Let's retrieve one of these.

In [9]:
template_id = matched[0]['@id']
template_id

'https://repo.metadatacenter.org/templates/916d64a9-e878-44e2-9d69-69eaf0bad233'

In [10]:
sess.get(template_id).json()

{'$schema': 'http://json-schema.org/draft-04/schema#',
 '@context': {'bibo': 'http://purl.org/ontology/bibo/',
  'oslc': 'http://open-services.net/ns/core#',
  'oslc:modifiedBy': {'@type': '@id'},
  'pav': 'http://purl.org/pav/',
  'pav:createdBy': {'@type': '@id'},
  'pav:createdOn': {'@type': 'xsd:dateTime'},
  'pav:lastUpdatedOn': {'@type': 'xsd:dateTime'},
  'schema': 'http://schema.org/',
  'schema:description': {'@type': 'xsd:string'},
  'schema:name': {'@type': 'xsd:string'},
  'xsd': 'http://www.w3.org/2001/XMLSchema#'},
 '@id': 'https://repo.metadatacenter.org/templates/916d64a9-e878-44e2-9d69-69eaf0bad233',
 '@type': 'https://schema.metadatacenter.org/core/Template',
 '_ui': {'order': ['Movie Type',
   'Lead Actor Family Name',
   'Lead Actor First Name'],
  'pages': [],
  'propertyLabels': {'Lead Actor Family Name': 'Lead Actor Family Name',
   'Lead Actor First Name': 'Lead Actor First Name',
   'Movie Type': 'Movie Type'}},
 'additionalProperties': False,
 'bibo:status': '

### Terminology Service
Do a search on the terminology service

In [11]:
term_search_params = {
    'q': 'melanoma',
    'scope': 'classes',
}
response = sess.get(terminology_endpoint + '/search', params=term_search_params)
response

<Response [200]>

In [12]:
data = response.json()
data.keys()

dict_keys(['nextPage', 'pageSize', 'prevPage', 'pageCount', 'page', 'collection'])

That's microservices for you - different paging model

In [13]:
len(data['collection'])

50

In [14]:
data['collection'][0:2]

[{'@id': 'http://purl.bioontology.org/ontology/MEDDRA/10053571',
  '@type': 'http://data.bioontology.org/metadata/OntologyClass',
  'definition': None,
  'id': 'http://purl.bioontology.org/ontology/MEDDRA/10053571',
  'prefLabel': 'Melanoma',
  'source': 'http://data.bioontology.org/ontologies/MEDDRA',
  'type': 'OntologyClass'},
 {'@id': 'http://purl.bioontology.org/ontology/LNC/LA14279-6',
  '@type': 'http://data.bioontology.org/metadata/OntologyClass',
  'definition': None,
  'id': 'http://purl.bioontology.org/ontology/LNC/LA14279-6',
  'prefLabel': 'Melanoma',
  'source': 'http://data.bioontology.org/ontologies/LOINC',
  'type': 'OntologyClass'}]

So, the terminology API works, but the paging model is very different. Things to note.